# Last Episode...

We learnt what an FPGA is
<img src="files/files/ecp5_block_diagram.png">

We learnt how to write combinatory and synchronous logic in migen

In [1]:
from migen import *

class mymodule(Module):
    def __init__(self, a, b, difference):
        # a and b must be Signal() instances
        y = Signal()
        y_ff = Signal()
        self.comb += [ # combinatorial: 'instantaneous'
            y.eq(a + b),
            difference.eq(a-b)
        ]
        self.sync += [ # synchronous: y_ff value is updated on next 'sys_clk' rising edge
            y_ff.eq(a+b)
        ]
        useless = Signal() # not used anywhere, won't generate logic

        self.y = y # will be acessible by parent Module
        # y_ff is not accessible by parent Module, thus, can't be connected to IOs or other modules.
        # it will be removed with its associated logic during the optimization phase

We learnt how to describe our system using litex's `Platform` and expand it

In [9]:
# the Platform represents the ulx3s 'capabilities', which FPGA brand/type is uses etc
from litex_boards.platforms.ulx3s import Platform
from litex.build.generic_platform import * # so we can use Pins(), etc to extend the platform

def buildplat():
    # by default, LiteX uses Lattice's closed-source toolchain.
    # ULX3S has 4 FPGA size variants, set 'device' accordingly
    plat = Platform(toolchain="trellis", device="LFE5U-12F")

    # these IOs should probably be defined by default in litex, but since it's not we have to do it
    plat.add_extension([
        ("user_button", 0, Pins("D6"), IOStandard("LVCMOS33")),  # BTN_PWRn (inverted logic)
        ("user_button", 1, Pins("R1"), IOStandard("LVCMOS33")),  # FIRE1
        ("user_button", 2, Pins("T1"), IOStandard("LVCMOS33")),  # FIRE2
        ("user_button", 3, Pins("R18"), IOStandard("LVCMOS33")), # UP
        ("user_button", 4, Pins("V1"), IOStandard("LVCMOS33")),  # DOWN
        ("user_button", 5, Pins("U1"), IOStandard("LVCMOS33")),  # LEFT
        ("user_button", 6, Pins("H16"), IOStandard("LVCMOS33")), # RIGHT
    ])
    return plat

ModuleNotFoundError: No module named 'litex_boards'

Then how to request pins, and build the system and load it onto the ULX3s

In [ ]:
# A bit of boilerplate...
from migen import * # the FHDL
from litex import * # the SoC builder

# DEMO 1: LED display state of push buttons
class hello(Module):
    def __init__(self, plat):
        self.platform = plat
        self.ios = set()
        for i in range(7):
            led = plat.request("user_led", i)
            btn = plat.request("user_button", i)
            self.comb += led.eq(btn)
            self.ios = self.ios | {led, btn}

plat = buildplat()
b = plat.build(hello(plat), run=True)
prog = plat.create_programmer().load_bitstream('build/top.bit')

# Now, let's make a SoC!

System on Chip = CPU + periperals (IOs, accelerators) + memory

<img src="files/files/precursor-soc-diagram.png">
https://www.crowdsupply.com/sutajio-kosagi/precursor/updates/a-guided-tour-of-the-precursor-system-on-chip-soc

## CPU
Litex gives a couple of CPUs to try from: vexrisc, lm32, minerva, mor1kx (and a lot more). Most of them use the Risc V ISA but you also find OPEN POWER, OpenRISC or other odball ISA.
All of them have a "wrapper" which typically contain a CPU-to-wishbone-interface, reset&interrupt signals.

We'll be using `vexriscv` CPU, as it's a good performance / size compromize and well supported by other projects like https://github.com/litex-hub/linux-on-litex-vexriscv

Its peripherals and memories will be connected through a Wishbone bus, which is a rather simple bus well suited to FPGAs SoC where typically only a few peripherals are connected.

# Memory
The ULX3s has a 32MB SDRAM onboard, which can be hooked to the Wishbone bus through a litedram core.

litex also creates a small ROM and RAM, which we will need to run the bootloader.

# Peripherals
litex provides plenty of useful peripherals, from basic ones like GPIO, UART, SPI, I2C, PWM to more complex peripherals like Ethernet, Framebuffer, PCIe endpoint, USB

We'll start with the basics:
* an UART, so we can have a serial terminal
* a timer
* a LED driver

We will also create our own peripheral using CSRs to control the LEDs (let's be honest, blinking lights is all that Electronics is for :)

# Most basic SoC
Fortunately, litex_boards provides an ou-of-the-box example for building an SoC on the ULX3s: https://github.com/litex-hub/litex-boards/blob/master/litex_boards/targets/ulx3s.py

It can be invoqued as a command-line tool. Run the following command (don't forget to source the virtualenv if you used one):  
`python -m litex_boards.targets.ulx3s --help`

We can build a complete SoC with this command (don't forget to change the device accordingly to the board you have):  
`python -m litex_boards.targets.ulx3s --device LFE5U-85F --build`

litex_board can be installed with this command: `pip3 install --user git+https://github.com/litex-hub/litex-boards.git`

## Other nice features
Litex can generate the documentation for your SoC. This will require `sphinx` and `spinxconfrib-wavdrom`
```
pip install sphinx spinxconfrib-wavdrom
python -m litex_boards.targets.ulx3s --device LFE5U-85F --no-compile-software --doc
python3 -m http.server --directory build/ulx3s/doc/_build/html 8080
```
Now open http://localhost:8080/

# Playing with our SoC
1. Load the SoC : `python -m litex_boards.targets.ulx3s --no-compile-software --load`
2. open a terminal:  `lxterm /dev/ttyUSB0` or `ujprog -t -P /dev/ttyUSB0` or `picocom -b 115200 /dev/ttyUSB0`
3. Try reading and writing a couple of CSR  
   `mem_write 0xf0000000 0`  
   `mem_write 0xf0002000 0x23`  
   `leds 3`

# Building and running application on the SoC
Once again, litex has some useful examples. A port of "lab004" firmware from https://github.com/litex-hub/fpga_101 is located in `lab004_firmware` directory.

You can build and load it by running:  
`cd lab004_firmware && make && lxterm /dev/ttyUSB0 --kernel firmware.bin`